# Using Experimental Simulators with Q# and Python

In [3]:
!pip install matplotlib
!pip install qutip

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 10.3 MB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 34.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 24.2 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 14.6 MB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 28.5 MB 39.3 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [1]:
import qutip as qt
import qsharp.experimental
qsharp.experimental.enable_noisy_simulation()

qsharp.config["experimental.simulators.nQubits"] = 1
qsharp.config["experimental.simulators.stabilizerStateStyle"] = "denseGroupPresentation"

Preparing Q# environment...
..

In [2]:
%%qsharp
open Microsoft.Quantum.Diagnostics;

operation DumpPlus() : Unit {
    use q = Qubit();
    H(q);
    DumpMachine();
    X(q);
    Reset(q);
}

## Depolarizing Noise

In [3]:
noise_model = qsharp.experimental.get_noise_model_by_name('ideal')

In [4]:
def depolarizing_noise(p=1.0):
    return p * qt.to_super(qt.qeye(2)) + ((1 - p) / 4) * sum(map(
        qt.to_super, [P.as_qobj() for P in qsharp.Pauli]
    ))

In [5]:
noise_model['h'] = depolarizing_noise(0.99) * qt.to_super(qt.qip.operations.hadamard_transform())
qsharp.experimental.set_noise_model(noise_model)

In [6]:
DumpPlus.simulate()

|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0.7071067811865476 + 0𝑖

()

In [7]:
DumpPlus.simulate_noise()

# of qubits,1
State data,$$ \left( \begin{matrix} 0.5001100693290156 + 0 i & 0.4955602658152263 + 0 i\\ 0.4955602658152263 + 0 i & 0.4995192803421991 + 0 i \end{matrix} \right) $$


()

## Stabilizer Simulation

In [8]:
noise_model = qsharp.experimental.get_noise_model_by_name('ideal_stabilizer')
noise_model['h'] = qsharp.experimental.SequenceProcess(1, 
    [
        qsharp.experimental.ChpDecompositionProcess(1, [
            qsharp.experimental.Hadamard(0)
        ]),
        qsharp.experimental.MixedPauliProcess(1, [
            (0.8, 'I'),
            (0.2, 'Z')
        ])
    ]
)
qsharp.experimental.set_noise_model(noise_model)
qsharp.config['experimental.simulators.representation'] = 'stabilizer'
qsharp.config['experimental.simulators.nQubits'] = 2

In [9]:
%%qsharp

operation DumpBellPair() : Unit {
    use left = Qubit();
    use right = Qubit();
    within {
        H(left);
        CNOT(left, right);
    } apply {
        Microsoft.Quantum.Diagnostics.DumpMachine();
    }
}

In [10]:
DumpBellPair.simulate()

|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0 + 0𝑖
|3⟩	0.7071067811865476 + 0𝑖

()

In [11]:
for _ in range(5):
    DumpBellPair.simulate_noise()

# of qubits,2
State data,"$$\left\langle -XX, ZZ \right\rangle$$"


# of qubits,2
State data,"$$\left\langle XX, ZZ \right\rangle$$"


# of qubits,2
State data,"$$\left\langle XX, ZZ \right\rangle$$"


# of qubits,2
State data,"$$\left\langle -XX, ZZ \right\rangle$$"


# of qubits,2
State data,"$$\left\langle XX, ZZ \right\rangle$$"
